group tasks:
<ul>
<li>Find if there is a correlation between the price of the apps and the category (Teen, Everyone, Mature).</li>
<li>find the sentiment of all apps using np files and "afinn" lib</li>
<li>for paid apps only list the top 5 highest and lowest sentiment numbers with the name of the app and the app category</li>
<li>what is the best category according to sentiment values</li>
</ul>

In [ ]:
!pip install afinn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53447 sha256=435a4a7251e681fb8d24a4310afb18b5a10a024bc72dba206996c006c12c7e99
  Stored in directory: /root/.cache/pip/wheels/f6/6f/c3/b305c5107a17618f2938a067d5ffcbb556909d82398762089e
Successfully built afinn


In [ ]:
import pandas as pd

ps = pd.read_csv('https://raw.githubusercontent.com/ufukcanadanir/playstore_db_team_1/main/cleaned_playstore.csv')
psr = pd.read_csv('https://raw.githubusercontent.com/ufukcanadanir/playstore_db_team_1/main/cleaned_playstore_reviews.csv')

n = pd.read_excel('https://github.com/ufukcanadanir/playstore_db_team_1/blob/main/n.xlsx?raw=true')
p = pd.read_excel('https://github.com/ufukcanadanir/playstore_db_team_1/blob/main/p.xlsx?raw=true')


In [ ]:
psr.head()

,Unnamed: 0,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333
1,1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,Great,Positive,0.800000,0.750000
2,2,11st,Horrible ID verification,Negative,-1.000000,1.000000
3,3,1800 Contacts - Lens Store,Great hassle free way order contacts. Got call...,Positive,0.600000,0.775000
4,4,1LINE – One Line with One Touch,"gets 1* there's ad every single level restart,...",Negative,-0.157143,0.704762


# Find if there is a correlation between the price of the apps and the category (Teen, Everyone, Mature).

In [ ]:
from scipy.stats import f_oneway

categories = ps.groupby(["Content Rating"])

names = [name for name, frame in categories]
prices = [frame["Price"] for name, frame in categories]

f_oneway(*prices)

F_onewayResult(statistic=0.5528435874888571, pvalue=0.7362752470096219)

Since the p-value is not less than 0.05 hence we could not reject the null hypothesis<a href="https://en.wikipedia.org/wiki/F-test#One-way_ANOVA_example"> (see)</a>. This implies that we do not have sufficient proof to say that there exists a difference in the prices among different categories. (at the %5 level of significance)

In [ ]:

paidonly = [df.where(df > 0).dropna() for df in prices if len(df.where(df > 0).dropna()) > 0]
f_oneway(*paidonly)

F_onewayResult(statistic=0.44347406472988726, pvalue=0.7219911725069318)

# find the sentiment of all apps using np files and "afinn" lib

In [ ]:
from afinn import Afinn

afinn = Afinn()
afinn_dict = afinn._dict

In [ ]:
new_dict = {}

for pw in p.values:
  new_dict[pw[0]] = 2

for nw in n.values:
  new_dict[nw[0]] = -2

new_dict = {**new_dict, **afinn_dict}

afinn._dict = new_dict

In [51]:
# conda install -c conda-forge  airflow
#afinn.score('This is utterly excellent!')

In [ ]:
psr["afinn"] = psr.Translated_Review.apply(lambda x: afinn.score(x))

In [ ]:
psr.groupby("App")["afinn"].mean()

App
10 Best Foods for You                                 13.0
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                       3.0
11st                                                  -3.0
1800 Contacts - Lens Store                             4.0
1LINE – One Line with One Touch                       -2.0
                                                      ... 
Hotels Combined - Cheap deals                         -3.0
Hotels.com: Book Hotel Rooms & Find Vacation Deals    -2.0
Hotspot Shield Free VPN Proxy & Wi-Fi Security        -1.0
Hotstar                                                0.0
Housing-Real Estate & Property                         2.0
Name: afinn, Length: 641, dtype: float64

# for paid apps only list the top 5 highest and lowest sentiment numbers with the name of the app and the app category

In [52]:
afinndf = psr.groupby("App")["afinn"].mean()
afinndf=pd.DataFrame(afinndf)
best = ps.join(afinndf, on='App')
best.dropna(subset=['afinn'], inplace=True)
best = best[best.Price != 0]


## Top 5 apps wit highest sentiment values

In [53]:
top5 = best.sort_values(by=['afinn'],ascending=True).head(5)
top5 =top5[['App','Category','afinn']]
top5

,App,Category,afinn
1761,Diabetes & Diet Tracker,MEDICAL,-4.0
1492,Farming Simulator 18,GAME,-3.0
2375,Golfshot Plus: Golf GPS,SPORTS,0.0
1494,Call of Duty:Black Ops Zombies,GAME,3.0
2673,Beautiful Widgets Pro,PERSONALIZATION,6.0


## Top 5 apps with lowest sentiment values

In [54]:
low5 = best.sort_values(by=['afinn'],ascending=False).head(5)
low5 =low5[['App','Category','afinn']]
low5

,App,Category,afinn
1754,Essential Anatomy 3,MEDICAL,12.0
2673,Beautiful Widgets Pro,PERSONALIZATION,6.0
2675,HD Widgets,PERSONALIZATION,6.0
2707,BIG Launcher,PERSONALIZATION,6.0
1494,Call of Duty:Black Ops Zombies,GAME,3.0


# what is the best category according to sentiment values

In [57]:
afinndf = psr.groupby("App")["afinn"].mean()
afinndf=pd.DataFrame(afinndf)
best = ps.join(afinndf, on='App')
best.dropna(subset=['afinn'], inplace=True)
#best.groupby('Category')['afinn'].mean()
best_category_accto_sent_values= best.groupby('Category')['afinn'].mean().sort_values(ascending=False).index[0]

In [58]:
best_category_accto_sent_values

'GAME'